In [124]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split

In [125]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from keras import layers
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, auc, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import pandas as pd
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [126]:
df = pd.read_csv('Preprocessing/fight_with_stats_precomp.csv')
df_fighters = pd.read_csv('Preprocessing/fighter_total_stats.csv')

In [127]:
df_fighters_details = pd.read_csv('Preprocessing/fighter_details.csv', parse_dates=True)

In [128]:
df = df[(df['fightNo_fighter']>=3) & (df['fightNo_opponent']>=3) & (df['Women_fight']==0)]

In [129]:
df = df.fillna(0)


In [130]:
df.head()

,EVENT,BOUT,FIGHTER,OPPONENT,METHOD,Win,Women_fight,ROUND,HEIGHT_diff,REACH_diff,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
24,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Patrick Smith,Royce Gracie,KO/TKO - ground,0,0,1,1.0,0.0,...,0.0,0.0,3.0,5.0,5.0,10.0,0.0,0.0,0.0,0.0
25,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Royce Gracie,Patrick Smith,KO/TKO - ground,1,0,1,-1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,3.0,5.0
26,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Remco Pardoel,Royce Gracie,Submission,0,0,1,2.0,0.0,...,0.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,0.0,2.0
27,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Royce Gracie,Remco Pardoel,Submission,1,0,1,-2.0,0.0,...,1.0,2.0,0.0,1.0,0.0,3.0,1.0,1.0,3.0,5.0
50,UFC 4: Revenge of the Warriors,Royce Gracie vs. Dan Severn,Dan Severn,Royce Gracie,Submission,0,0,1,1.0,0.0,...,3.0,4.0,2.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0


In [131]:
df.shape

(6837, 63)

In [132]:
df.columns.tolist()[19:]

['KD_fighter',
 'SUB.ATT_fighter',
 'REV._fighter',
 'CTRL_fighter',
 'SIG.STR._landed_fighter',
 'SIG.STR._attemps_fighter',
 'TOTAL STR._landed_fighter',
 'TOTAL STR._attemps_fighter',
 'TD_landed_fighter',
 'TD_attemps_fighter',
 'HEAD_landed_fighter',
 'HEAD_attemps_fighter',
 'BODY_landed_fighter',
 'BODY_attemps_fighter',
 'LEG_landed_fighter',
 'LEG_attemps_fighter',
 'DISTANCE_landed_fighter',
 'DISTANCE_attemps_fighter',
 'CLINCH_landed_fighter',
 'CLINCH_attemps_fighter',
 'GROUND_landed_fighter',
 'GROUND_attemps_fighter',
 'KD_opponent',
 'SUB.ATT_opponent',
 'REV._opponent',
 'CTRL_opponent',
 'SIG.STR._landed_opponent',
 'SIG.STR._attemps_opponent',
 'TOTAL STR._landed_opponent',
 'TOTAL STR._attemps_opponent',
 'TD_landed_opponent',
 'TD_attemps_opponent',
 'HEAD_landed_opponent',
 'HEAD_attemps_opponent',
 'BODY_landed_opponent',
 'BODY_attemps_opponent',
 'LEG_landed_opponent',
 'LEG_attemps_opponent',
 'DISTANCE_landed_opponent',
 'DISTANCE_attemps_opponent',
 'CLINCH

In [133]:
df = df.merge(df[['EVENT', 'BOUT', 'FIGHTER']+df.columns.tolist()[15:]] ,how='left', left_on=['EVENT', 'BOUT', 'OPPONENT'], right_on=['EVENT', 'BOUT', 'FIGHTER'], suffixes=('_f1', '_f2'))

In [134]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6837 entries, 0 to 6836
Data columns (total 112 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    EVENT                           6837 non-null   object 
 1    BOUT                            6837 non-null   object 
 2    FIGHTER_f1                      6837 non-null   object 
 3    OPPONENT                        6837 non-null   object 
 4    METHOD                          6837 non-null   object 
 5    Win                             6837 non-null   int64  
 6    Women_fight                     6837 non-null   int64  
 7    ROUND                           6837 non-null   int64  
 8    HEIGHT_diff                     6837 non-null   float64
 9    REACH_diff                      6837 non-null   float64
 10   AGE_diff                        6837 non-null   float64
 11   form_skore_fighter              6837 non-null   float64
 12   form_skore_opponen

In [12]:
df = df.drop('FIGHTER_f2',axis=1)

In [13]:
'LEG_attemps_fighter_f1'.split('_')

['LEG', 'attemps', 'fighter', 'f1']

In [14]:
sloupce1 = df.columns.tolist()[19:]
vynechat = ['Win_tot_f2', 'DRAW_f2','No_contest_f2']
sloupce = [x for x in sloupce1 if x not in vynechat]
for sloupec in sloupce:    
    splited = sloupec.split('_')
    if 'CTRL' in splited:
        df.loc[:,splited[0]+'_pct_'+splited[1]+splited[2]] = df[sloupec]/df['TotalTime'+'_'+splited[2]]
    if 'attemps' in splited:
        df.loc[:,splited[0]+'_acc_'+ splited[2] + splited[3]] = df[sloupec.replace('attemps', 'landed')]/df[sloupec]
        df.loc[:,splited[0]+'_perRound_'+ splited[2] + splited[3]] = (df[sloupec.replace('attemps', 'landed')]/df['TotalTime'+'_'+splited[3]])*300
        
    

In [15]:
df.iloc[:,:5]

,EVENT,BOUT,FIGHTER_f1,OPPONENT,METHOD
0,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Patrick Smith,Royce Gracie,KO/TKO - ground
1,UFC 2: No Way Out,Royce Gracie vs. Patrick Smith,Royce Gracie,Patrick Smith,KO/TKO - ground
2,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Remco Pardoel,Royce Gracie,Submission
3,UFC 2: No Way Out,Royce Gracie vs. Remco Pardoel,Royce Gracie,Remco Pardoel,Submission
4,UFC 4: Revenge of the Warriors,Royce Gracie vs. Dan Severn,Dan Severn,Royce Gracie,Submission
...,...,...,...,...,...
6832,UFC 283: Teixeira vs. Hill,Paul Craig vs. Johnny Walker,Paul Craig,Johnny Walker,KO/TKO - clinch
6833,UFC 283: Teixeira vs. Hill,Shamil Abdurakhimov vs. Jailton Almeida,Jailton Almeida,Shamil Abdurakhimov,KO/TKO - ground
6834,UFC 283: Teixeira vs. Hill,Shamil Abdurakhimov vs. Jailton Almeida,Shamil Abdurakhimov,Jailton Almeida,KO/TKO - ground
6835,UFC 283: Teixeira vs. Hill,Warlley Alves vs. Nicolas Dalby,Nicolas Dalby,Warlley Alves,Decision


In [ ]:
df['Avg_time1'] = df['TotalTime_f1']/df['fightNo_fighter']
df['Avg_time2'] = df['TotalTime_f2']/df['fightNo_opponent']

In [16]:
df = df.drop(sloupce, axis=1)

In [17]:
df_model = df.drop(['EVENT','BOUT','FIGHTER_f1','OPPONENT','METHOD','Women_fight','TotalTime_f1', 'Win'],axis=1)

In [18]:
df_model = df_model.fillna(0)
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6837 entries, 0 to 6836
Data columns (total 91 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Win                             6837 non-null   int64  
 1   ROUND                           6837 non-null   int64  
 2   HEIGHT_diff                     6837 non-null   float64
 3   REACH_diff                      6837 non-null   float64
 4   AGE_diff                        6837 non-null   float64
 5   form_skore_fighter              6837 non-null   float64
 6   form_skore_opponent             6837 non-null   float64
 7   fightNo_fighter                 6837 non-null   int64  
 8   fightNo_opponent                6837 non-null   float64
 9   Win_tot_f1                      6837 non-null   float64
 10  DRAW_f1                         6837 non-null   float64
 11  No_contest_f1                   6837 non-null   float64
 12  Win_tot_f2                      68

In [98]:
X=df_model.loc[:, ~df_model.columns.isin(['ROUND'])]
y=df_model['ROUND']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 0)
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(max_iter=50000)
logreg.fit(X_train,y_train)
predictions = logreg.predict(X_test)
acc = accuracy_score(y_test, predictions) 
f1 = f1_score(y_test, predictions,average='micro')
print(f1, acc)

0.5023391812865498 0.5023391812865498


In [111]:
X_new.shape

(6637, 200, 90)

In [112]:
import pandas as pd
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras
# načtení dat

# definice lookback
lookback = 200

# převod dat na numpy array
data = X.values
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# rozdělení dat na trénovací a testovací
X_new = []
y_new = []
for i in range(lookback, len(data)):
    X_new.append(data[i-lookback:i])
    y_new.append(y[i])
X_new= np.array(X_new)
y_new = np.array(y_new)
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2)
train_y_one_hot = keras.utils.to_categorical(y_train)
test_y_one_hot = keras.utils.to_categorical(y_test)

# vytvoření modelu
model = Sequential()
model.add(LSTM(units=100, input_shape=(lookback, X_new.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(6, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

# natrénování modelu na trénovacích datech
model.fit(X_train, train_y_one_hot, epochs=100, batch_size=32,validation_data=(X_test, test_y_one_hot))

# hodnocení modelu na testovacích datech
test_loss, test_acc = model.evaluate(X_test, test_y_one_hot)
print('Test accuracy:', test_acc)


(6837, 90)
Epoch 1/100
166/166 [==============================] - 19s 107ms/step - loss: 1.2862 - accuracy: 0.4590 - val_loss: 1.2171 - val_accuracy: 0.4947
Epoch 2/100
166/166 [==============================] - 17s 104ms/step - loss: 1.2414 - accuracy: 0.4890 - val_loss: 1.2025 - val_accuracy: 0.4992
Epoch 3/100
166/166 [==============================] - 17s 105ms/step - loss: 1.2302 - accuracy: 0.4941 - val_loss: 1.2039 - val_accuracy: 0.5000
Epoch 4/100
166/166 [==============================] - 17s 105ms/step - loss: 1.2202 - accuracy: 0.5014 - val_loss: 1.2046 - val_accuracy: 0.4985
Epoch 5/100
166/166 [==============================] - 17s 104ms/step - loss: 1.2128 - accuracy: 0.5042 - val_loss: 1.2115 - val_accuracy: 0.4970
Epoch 6/100
166/166 [==============================] - 17s 104ms/step - loss: 1.2047 - accuracy: 0.5052 - val_loss: 1.2027 - val_accuracy: 0.4970
Epoch 7/100
166/166 [==============================] - 17s 103ms/step - loss: 1.2043 - accuracy: 0.5042 - val_los

Epoch 57/100
166/166 [==============================] - 17s 102ms/step - loss: 1.0281 - accuracy: 0.5785 - val_loss: 1.2596 - val_accuracy: 0.4834
Epoch 58/100
166/166 [==============================] - 17s 102ms/step - loss: 1.0272 - accuracy: 0.5766 - val_loss: 1.2435 - val_accuracy: 0.4706
Epoch 59/100
166/166 [==============================] - 17s 102ms/step - loss: 1.0157 - accuracy: 0.5754 - val_loss: 1.2577 - val_accuracy: 0.4797
Epoch 60/100
166/166 [==============================] - 17s 102ms/step - loss: 1.0131 - accuracy: 0.5785 - val_loss: 1.2501 - val_accuracy: 0.4849
Epoch 61/100
166/166 [==============================] - 17s 102ms/step - loss: 1.0050 - accuracy: 0.5847 - val_loss: 1.2657 - val_accuracy: 0.4714
Epoch 62/100
166/166 [==============================] - 17s 102ms/step - loss: 1.0128 - accuracy: 0.5850 - val_loss: 1.2596 - val_accuracy: 0.4774
Epoch 63/100
166/166 [==============================] - 17s 103ms/step - loss: 0.9974 - accuracy: 0.5882 - val_loss: 1

In [113]:
X_test.shape

(1328, 200, 90)

In [114]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
y_pred_proba =model.predict(X_test)
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.title("AUC ROC Curve")
plt.show()

42/42 [==============================] - 2s 42ms/step


ValueError: multiclass format is not supported

In [ ]:
from sklearn.metrics import classification_report
auc = roc_auc_score(y_test, y_pred_proba) 
y_pred_binary = (y_pred_proba > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))
print('AUC-ROC score: ', auc)

In [ ]:
np.mean(y_pred_proba)

In [ ]:
model.save('model_round.h5')
model.save_weights('model_round.h5')  # to store


In [ ]:
new_data  =data_scaled[50,:]
new_data = new_data[1:]
new_data

In [ ]:

new_data = np.reshape(new_data, (1,1,new_data.shape[0]))
new_data

In [ ]:
y_pred = model.predict(new_data)

In [ ]:
y_pred

In [115]:
from datetime import date

today = date.today()
df_fighters_details['AGE'] = (pd.to_datetime(today) - pd.to_datetime(df_fighters_details['DOB'])).astype('<m8[Y]')

In [116]:
fighter1 = "Derrick Lewis"
fighter2= "Serghei Spivac"


In [117]:
f1_df = df_fighters.loc[df_fighters['FIGHTER']==fighter1]
f2_df = df_fighters.loc[df_fighters['FIGHTER']==fighter2]

In [118]:
f1_df

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Fights,Win,DRAW,Lost,No_contest,...,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent,form_skore_fighter
597,Derrick Lewis,75.0,260.0,79.0,Orthodox,25,17,0,8,0,...,179.0,115.0,130.0,365.0,694.0,105.0,148.0,110.0,146.0,-0.7


In [119]:
df_model.columns

Index(['Win', 'ROUND', 'HEIGHT_diff', 'REACH_diff', 'AGE_diff',
       'form_skore_fighter', 'form_skore_opponent', 'fightNo_fighter',
       'fightNo_opponent', 'Win_tot_f1', 'DRAW_f1', 'No_contest_f1',
       'Win_tot_f2', 'DRAW_f2', 'No_contest_f2', 'CTRL_pct_fighterf1',
       'SIG.STR._acc_fighterf1', 'SIG.STR._perRound_fighterf1',
       'TOTAL STR._acc_fighterf1', 'TOTAL STR._perRound_fighterf1',
       'TD_acc_fighterf1', 'TD_perRound_fighterf1', 'HEAD_acc_fighterf1',
       'HEAD_perRound_fighterf1', 'BODY_acc_fighterf1',
       'BODY_perRound_fighterf1', 'LEG_acc_fighterf1',
       'LEG_perRound_fighterf1', 'DISTANCE_acc_fighterf1',
       'DISTANCE_perRound_fighterf1', 'CLINCH_acc_fighterf1',
       'CLINCH_perRound_fighterf1', 'GROUND_acc_fighterf1',
       'GROUND_perRound_fighterf1', 'CTRL_pct_opponentf1',
       'SIG.STR._acc_opponentf1', 'SIG.STR._perRound_opponentf1',
       'TOTAL STR._acc_opponentf1', 'TOTAL STR._perRound_opponentf1',
       'TD_acc_opponentf1', 'TD_

In [ ]:
agediff = df_fighters_details[df_fighters_details['FIGHTER']==fighter1]['AGE'].values[0] - df_fighters_details[df_fighters_details['FIGHTER']==fighter2]['AGE'].values[0]
formy = [f1_df['form_skore_fighter'].values[0], f2_df['form_skore_fighter'].values[0]]
no_of_fights = [f1_df['Fights'].values[0],f2_df['Fights'].values[0]]
W_D_NC = f1_df[['Win','DRAW', 'No_contest']].values.tolist()[0] + f2_df[['Win','DRAW', 'No_contest']].values.tolist()[0]

In [ ]:
sloupce2 = df_fighters.columns.tolist()[10:]
stats_f1 = []
stats_f2 = []
for sloupec in sloupce2:    
    splited = sloupec.split('_')
    if 'CTRL' in splited:
        stats_f1.append((f1_df[sloupec]/f1_df['TotalTime']).values[0])
        stats_f2.append((f2_df[sloupec]/f2_df['TotalTime']).values[0])
    if 'attemps' in splited:
        stats_f1.append((f1_df[sloupec.replace('attemps', 'landed')]/f1_df[sloupec]).values[0])
        stats_f1.append((f1_df[sloupec.replace('attemps', 'landed')]/f1_df['TotalTime']).values[0]*300)
        
        stats_f2.append((f2_df[sloupec.replace('attemps', 'landed')]/f2_df[sloupec]).values[0])
        stats_f2.append((f2_df[sloupec.replace('attemps', 'landed')]/f2_df['TotalTime']).values[0]*300)
stats_list = stats_f1 + stats_f2

In [ ]:
vstup = np.array([1]+[f1_df.iloc[0][col] - f2_df.iloc[0][col] for col in ['HEIGHT_fighter','REACH_fighter']] + [agediff] + formy + no_of_fights + W_D_NC + stats_list)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
vstup_scaled = scaler.fit_transform(df_model.append(pd.DataFrame(vstup.reshape(1,-1), columns=list(df_model)), ignore_index=True))[-200:,1:]

In [ ]:
vstup_scaled.shape

In [ ]:
from numpy import *
where_are_NaNs = isnan(vstup_scaled)
vstup_scaled[where_are_NaNs] = 0

In [ ]:
np.argwhere(np.isnan(vstup_scaled))

In [ ]:
new_data = np.reshape(vstup_scaled, (1,200,vstup_scaled.shape[1]))

In [ ]:
new_data

In [ ]:
from keras.models import load_model
model = load_model('model.h5')
model.load_weights('my_model_weights.h5')  # to load

In [ ]:
y_pred = model.predict(new_data)
y_pred

In [ ]:
df_model.to_csv('df_model.csv',index=False)